## Deliverable 2. Create a Customer Travel Destinations Map.

In [4]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps
import ipywidgets as widgets

# Import API key
from config import map_key

# Configure gmaps API key
gmaps.configure(api_key=map_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Seoul,KR,37.5683,126.9778,49.44,82,0,3.44,clear sky
1,1,Sinnamary,GF,5.3833,-52.9500,77.25,89,97,9.60,overcast clouds
2,2,Severo-Kurilsk,RU,50.6789,156.1250,32.65,87,95,2.30,overcast clouds
3,3,Hithadhoo,MV,-0.6000,73.0833,82.89,73,99,4.50,overcast clouds
4,4,Inta,RU,66.0317,60.1659,30.49,98,100,11.18,light snow


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_temp = float(input('What is the maximum temperature you would like for your trip? '))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & \
                                       (city_data_df['Max Temp'] >= min_temp)]
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Sinnamary,GF,5.3833,-52.9500,77.25,89,97,9.60,overcast clouds
3,3,Hithadhoo,MV,-0.6000,73.0833,82.89,73,99,4.50,overcast clouds
7,7,Mumford,GH,5.2618,-0.7590,78.21,80,100,5.08,overcast clouds
8,8,Rikitea,PF,-23.1203,-134.9692,78.57,70,7,14.45,clear sky
15,15,Mitsamiouli,KM,-11.3847,43.2844,80.83,75,11,4.14,few clouds
...,...,...,...,...,...,...,...,...,...,...
672,672,Conde,BR,-7.2597,-34.9075,76.71,82,71,8.79,broken clouds
673,673,Veraval,IN,20.9000,70.3667,79.38,71,0,14.83,clear sky
678,678,Hoi An,VN,15.8794,108.3350,76.96,89,62,7.11,broken clouds
679,679,Saint-Leu,RE,-21.1500,55.2833,77.83,82,2,2.42,clear sky


In [9]:
# 4a. Determine if there are any empty rows.
clean_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = clean_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Sinnamary,GF,5.3833,-52.9500,77.25,89,97,9.60,overcast clouds
3,3,Hithadhoo,MV,-0.6000,73.0833,82.89,73,99,4.50,overcast clouds
7,7,Mumford,GH,5.2618,-0.7590,78.21,80,100,5.08,overcast clouds
8,8,Rikitea,PF,-23.1203,-134.9692,78.57,70,7,14.45,clear sky
15,15,Mitsamiouli,KM,-11.3847,43.2844,80.83,75,11,4.14,few clouds
...,...,...,...,...,...,...,...,...,...,...
672,672,Conde,BR,-7.2597,-34.9075,76.71,82,71,8.79,broken clouds
673,673,Veraval,IN,20.9000,70.3667,79.38,71,0,14.83,clear sky
678,678,Hoi An,VN,15.8794,108.3350,76.96,89,62,7.11,broken clouds
679,679,Saint-Leu,RE,-21.1500,55.2833,77.83,82,2,2.42,clear sky


In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
clean_hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
clean_hotel_df["Hotel Name"] = ""
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Sinnamary,GF,77.25,overcast clouds,5.3833,-52.9500,
3,Hithadhoo,MV,82.89,overcast clouds,-0.6000,73.0833,
7,Mumford,GH,78.21,overcast clouds,5.2618,-0.7590,
8,Rikitea,PF,78.57,clear sky,-23.1203,-134.9692,
15,Mitsamiouli,KM,80.83,few clouds,-11.3847,43.2844,
17,Vila Velha,BR,76.95,few clouds,-20.3297,-40.2925,
20,Vaini,TO,84.36,broken clouds,-21.2000,-175.2000,
23,Avarua,CK,82.45,broken clouds,-21.2078,-159.7750,
30,Kavieng,PG,85.24,overcast clouds,-2.5744,150.7967,
32,Hilo,US,80.28,overcast clouds,19.7297,-155.0900,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": map_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in clean_hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        clean_hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except (IndexError):
        clean_hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
        

In [13]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = clean_hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Sinnamary,GF,77.25,overcast clouds,5.3833,-52.9500,ALETHEIA Appartement Sinnamary
3,Hithadhoo,MV,82.89,overcast clouds,-0.6000,73.0833,Scoop Guest House
7,Mumford,GH,78.21,overcast clouds,5.2618,-0.7590,Spanish Villa
8,Rikitea,PF,78.57,clear sky,-23.1203,-134.9692,People ThankYou
15,Mitsamiouli,KM,80.83,few clouds,-11.3847,43.2844,Foyer ADM
...,...,...,...,...,...,...,...
672,Conde,BR,76.71,broken clouds,-7.2597,-34.9075,Pousada Beija Flor
673,Veraval,IN,79.38,clear sky,20.9000,70.3667,Lords Inn Somnath
678,Hoi An,VN,76.96,broken clouds,15.8794,108.3350,Belle Maison Hadana Hoi An managed by H&K Hosp...
679,Saint-Leu,RE,77.83,clear sky,-21.1500,55.2833,La Caz Oceane


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.4)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))